In [1]:
import os
import re

import itertools

In [2]:
# Перевод из алфавита base64
ALPHABET = 'ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789+/'
to_int = lambda char: ALPHABET.find(char)

#разбиение на массивы длины step
split_by_step = lambda l, step: [l[index: index+step] for index in range(0, len(l), step)]

# Функция склейки битовой карты из массива чисел с округлением битов до value
to_bitmap = lambda text, values: ''.join([f"{char:0{values}b}" for char in text])

# Функция разделяет по count массивам, где каждый элемент добавляется с шагом count
split_on_count = lambda l, count: [l[index-1::count] for index in range(1, count+1)]

# Регулярное выражение для символов
symbols_finder_re = re.compile("[^a-zA-Z0-9?!\-_.,:;'\n\t\v ]")

In [3]:
def decode_base64(data):
    # Переводим из base64
    added_chars = data.count('=')
    clean_data = data[:-added_chars]
    codes = [to_int(char) for char in clean_data if char != '=']
    bitmap = to_bitmap(codes, 6)
    bitmap = bitmap[:(-added_chars) * 2]
    char_bytes = split_by_step(bitmap, 8)
    # Переводим в числа для XOR'а
    int_hex = [int(bits, 2) for bits in char_bytes]
    return int_hex

In [4]:
def decode_message(text, key, return_symbols=False):
    # Дополняем ключ до размера сообщения и все xor-им
    if isinstance(key, str):
        keys = [ord(k) for k in key]
    elif not isinstance(key, list):
        keys = [key]
    else:
        keys = key

    length_input = len(text)
    
    div_length = length_input // len(keys)
    mod_length = length_input % len(keys)

    key_bits_all_length = keys * div_length + keys[:mod_length]
    x_o_r = [text[index] ^ key_bits_all_length[index] for index in range(length_input)]

    if return_symbols:
        return ''.join([chr(c) for c in x_o_r])
    else:
        return x_o_r

In [5]:
def decrypt_string(text):
    # Подбираем символ для ключа из диапазона до 128-го ASCI
    ascii_codes = list(range(0, 128))
    results = list()
    for key in ascii_codes:
        decoded = decode_message(text, key, return_symbols=True)
        checked_str = re.sub(symbols_finder_re, '', decoded)
        if len(checked_str) == len(text):
            results.append(key)
    return results

In [6]:
split_on_count(['20', '48', 'AF'], 2) ## проверка функции

[['20', 'AF'], ['48']]

In [7]:
decoded_files = list()
files = [file for file in os.listdir('.') if file.startswith('task16') and file.endswith('.input')]

for file in files:
    with open(file, 'r') as f:
        input_data = f.read()
        input_data = input_data.replace('\n', '')
    #if input_data.endswith('\n'):
    #    input_data = input_data[:-1]
    int_hex = decode_base64(input_data)
    
    key_length = 2
    while True:
        splitted_data = split_on_count(int_hex, key_length)

        #keys = ''
        keys = list()
        for data in splitted_data:
            res = decrypt_string(data)
            if res:
                keys.append(res)
            else:
                break

        if len(keys) == key_length:
            key = ''.join([chr(k[0]) for k in keys])
            message = decode_message(int_hex, key, return_symbols=True)
            decoded_files.append((file, key, message))
            break

        key_length += 1        
        if key_length == len(int_hex) // 10:
            print('Перебор ключей закончен')
            break

In [8]:
file, key, message = decoded_files[0]
print(f"Файл: {file}")
print(f"Ключ: {key}")
print()
print(f"Сообщение: {message}")

Файл: task160.input
Ключ: Robert Laurence Binyon

Сообщение: How dark, how quiet sleeps the vale below!
In the dim farms, look, not a window shines:
Distantly heard among the lonely pines,
How soft the languid autumn breezes flow
Past me, and kiss my hair, and cheek, and mouth!
Half--veiled is the calm sky:
Jupiter's kingly eye
Alone glows full in the unclouded South.

Alas! and can sweet Night avail to heal
Not one of the world's wounds? Must I, even here,
Still listen with the mind's too wakeful ear
To that sad sound, which in my flesh I feel;
Sound of unresting, unrejoicing feet,
With feverish steps or slow
For ever, to and fro,
Pacing the gay, thronged, friendless, stony street?

Nature is free; but Man the eternal slave
Of care and passion. Must I deem that true?
With fields and quiet have we nought to do,
Because our spirits for ever crave and crave,
And never found their satisfaction yet?
World, is thy heart so cold,
So deeply weary and old,
That thy sole business is but to forg

In [9]:
file, key, message = decoded_files[1]
print(f"Файл: {file}")
print(f"Ключ: {key}")
print()
print(f"Сообщение: {message}")

Файл: task161.input
Ключ: Terminator X: Bring the noise

Сообщение: I'm back and I'm ringin' the bell 
A rockin' on the mike while the fly girls yell 
In ecstasy in the back of me 
Well that's my DJ Deshay cuttin' all them Z's 
Hittin' hard and the girlies goin' crazy 
Vanilla's on the mike, man I'm not lazy. 

I'm lettin' my drug kick in 
It controls my mouth and I begin 
To just let it flow, let my concepts go 
My posse's to the side yellin', Go Vanilla Go! 

Smooth 'cause that's the way I will be 
And if you don't give a damn, then 
Why you starin' at me 
So get off 'cause I control the stage 
There's no dissin' allowed 
I'm in my own phase 
The girlies sa y they love me and that is ok 
And I can dance better than any kid n' play 

Stage 2 -- Yea the one ya' wanna listen to 
It's off my head so let the beat play through 
So I can funk it up and make it sound good 
1-2-3 Yo -- Knock on some wood 
For good luck, I like my rhymes atrocious 
Supercalafragilisticexpialidocious 
I'm an ef